<h1>Content-Aware Video Cropping (Batch Videos at a time from particular folder location)</h1>

In [ ]:
!pip install ultralytics
!pip install moviepy
!pip install -q supervision[assets] jupyter_bbox_widget

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.7/213.7 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.5/181.5 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.4/727.4 kB 36.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 77.0 MB/s eta 0:00:00


In [ ]:
from ultralytics import YOLO
import supervision as sv
import cv2
import os
import numpy as np
import torch
import json
from pathlib import Path
import tempfile
from google.colab import drive
import shutil
from moviepy.editor import VideoFileClip

drive.mount('/content/drive')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Mounted at /content/drive


<h2>For reproducing the results, make sure to mount the correct drive, and then change the part according to the location of dataset</h2>

In [ ]:
MODEL_NAME = 'yolo11n-seg.pt'
BATCH_INPUT_PATH = '/content/drive/MyDrive/Genuin Assignment/double_class_ip'
BATCH_OUTPUT_PATH = '/content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip'
DETECTIONS_FILE_PATH = '/content/detections.jsonl'
OUTPUT_WIDTH = 1080
OUTPUT_HEIGHT = 1920
# Batch processing parameters
BATCH_SIZE = 128  # Reduced batch size for memory efficiency
TARGET_SIZE = (640, 640)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

Using device: cuda


In [ ]:
model = YOLO(MODEL_NAME)
model.to(device)

100%|██████████| 5.90M/5.90M [00:00<00:00, 96.7MB/s]


YOLO(
  (model): SegmentationModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_runni

In [ ]:
def save_detection(detection_data, file_path):
    with open(file_path, 'a') as f:
        json.dump(detection_data, f)
        f.write('\n')

def process_frame_batch(frames, indices, cap_props):
    width, height = int(cap_props['width']), int(cap_props['height'])
    processed_frames = []

    # Process each frame
    for frame in frames:
        # Resize and preprocess
        resized_frame = cv2.resize(frame, TARGET_SIZE)
        frame_rgb = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2RGB)
        frame_normalized = frame_rgb.astype(np.float32) / 255.0
        processed_frames.append(frame_normalized)

    # Convert to tensor
    frames_array = np.stack(processed_frames)
    frames_tensor = torch.from_numpy(frames_array).permute(0, 3, 1, 2)

    # Free up memory
    del processed_frames, frames_array
    torch.cuda.empty_cache()

    # Run detection
    results = model(frames_tensor, stream=True)

    # Process results and save
    for idx, (result, orig_frame) in enumerate(zip(results, frames)):
        frame_num = indices[idx]

        # Scale factors
        scale_x = width / TARGET_SIZE[0]
        scale_y = height / TARGET_SIZE[1]

        # Create detection data
        frame_detections = {
            'frame_number': frame_num,
            'detections': []
        }

        if hasattr(result, 'boxes') and len(result.boxes) > 0:
            for box, cls in zip(result.boxes.xyxy, result.boxes.cls):
                # Scale coordinates
                scaled_box = [
                    float(box[0].item() * scale_x),
                    float(box[1].item() * scale_y),
                    float(box[2].item() * scale_x),
                    float(box[3].item() * scale_y)
                ]

                detection = {
                    'class': result.names[int(cls)],
                    'confidence': float(box.conf) if hasattr(box, 'conf') else None,
                    'bbox': scaled_box
                }

                frame_detections['detections'].append(detection)

        # Save detection data
        save_detection(frame_detections, DETECTIONS_FILE_PATH)

def process_video(MAIN_INPUT_PATH):
    # Open video capture
    cap = cv2.VideoCapture(MAIN_INPUT_PATH)

    # Get video properties
    cap_props = {
        'width': cap.get(cv2.CAP_PROP_FRAME_WIDTH),
        'height': cap.get(cv2.CAP_PROP_FRAME_HEIGHT),
        'fps': cap.get(cv2.CAP_PROP_FPS)
    }

    frames_batch = []
    frame_indices = []
    frame_count = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frames_batch.append(frame)
        frame_indices.append(frame_count)

        if len(frames_batch) == BATCH_SIZE:
            try:
              process_frame_batch(frames_batch, frame_indices, cap_props)
            except Exception as e:
                print(f"Error processing batch at frame {frame_count}: {str(e)}")

            # Clear batch
            frames_batch = []
            frame_indices = []
            torch.cuda.empty_cache()  # Clear CUDA cache

        frame_count += 1
        if frame_count % 10 == 0:
            print(f"Processed frame {frame_count}")

    # Process remaining frames
    if frames_batch:
        try:
          process_frame_batch(frames_batch, frame_indices, cap_props)
        except Exception as e:
            print(f"Error processing final batch: {str(e)}")

    # Release resources
    cap.release()

    print(f"Detection data saved")

def load_detections():
    detections = []
    with open(DETECTIONS_FILE_PATH, 'r') as f:
        for line in f:
            detections.append(json.loads(line))
    return detections

In [ ]:
def transform_jsonl(input_file):
    transformed_data = []

    # Open and read the JSONL file
    with open(input_file, 'r') as file:
        for line in file:
            # Parse each line as JSON
            frame_data = json.loads(line.strip())
            frame_number = frame_data["frame_number"]
            bounding_boxes = [detection["bbox"] for detection in frame_data["detections"]]
            transformed_data.append({"frame_number": frame_number, "bounding_boxes": bounding_boxes})

    return transformed_data

In [ ]:
def calculate_916_crop(frame_data, image_width, image_height):
    # Extract bounding boxes from the frame data
    bounding_boxes = frame_data.get("bounding_boxes", [])

    # Define 9:16 target ratio
    target_ratio = 9 / 16

    # Determine fixed crop dimensions based on video size
    if image_width / image_height > target_ratio:
        # Height is limiting factor
        crop_height = image_height
        crop_width = crop_height * target_ratio
    else:
        # Width is limiting factor
        crop_width = image_width
        crop_height = crop_width / target_ratio

    # Add padding if desired (e.g., 10%)
    padding_factor = 1.1
    crop_width *= padding_factor
    crop_height *= padding_factor

    # Ensure crop doesn't exceed image dimensions
    crop_width = min(crop_width, image_width)
    crop_height = min(crop_height, image_height)

    # Handle bounding_boxes
    if not bounding_boxes:
        center_x, center_y = image_width / 2, image_height / 2
    else:
        # Find the largest object (by area) in the bounding boxes
        largest_box = max(
            bounding_boxes,
            key=lambda box: (box[2] - box[0]) * (box[3] - box[1])
        )
        box_width = largest_box[2] - largest_box[0]
        box_height = largest_box[3] - largest_box[1]

        # Determine if the box is too large for the crop dimensions
        if box_width > crop_width or box_height > crop_height:
            # Focus on the center of the box
            center_x = (largest_box[0] + largest_box[2]) / 2
            center_y = (largest_box[1] + largest_box[3]) / 2
        else:
            # Focus on the top-left corner of the box
            center_x = largest_box[0]
            center_y = largest_box[1]

    # Calculate initial crop coordinates
    x1 = max(0, center_x - crop_width / 2)
    y1 = max(0, center_y - crop_height / 2)
    x2 = x1 + crop_width
    y2 = y1 + crop_height

    # Adjust crop if it exceeds boundaries
    if x2 > image_width:
        x2 = image_width
        x1 = x2 - crop_width
    if y2 > image_height:
        y2 = image_height
        y1 = y2 - crop_height
    if x1 < 0:
        x1 = 0
        x2 = crop_width
    if y1 < 0:
        y1 = 0
        y2 = crop_height

    # Ensure crop coordinates stay within the image boundaries
    x1 = max(0, x1)
    y1 = max(0, y1)
    x2 = min(image_width, x2)
    y2 = min(image_height, y2)

    return [int(x1), int(y1), int(x2), int(y2)]


In [ ]:
def cut_video_with_smoothing(video_path, detections_list, output_path, output_width=1080, output_height=1920, smoothing_factor=0.95):
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (output_width, output_height))

    frame_count = 0
    color = (0, 255, 0)
    thickness = 2

    # Initialize previous crop coordinates
    previous_crop_coords = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count < len(detections_list):
            boxes = detections_list[frame_count]
            # Calculate current crop coordinates
            current_crop_coords = calculate_916_crop(boxes, width, height)

            if previous_crop_coords is None:
                # First frame, no smoothing applied
                smoothed_crop_coords = current_crop_coords
            else:
                # Apply exponential smoothing
                smoothed_crop_coords = [
                    int(previous_crop_coords[i] * smoothing_factor + current_crop_coords[i] * (1 - smoothing_factor))
                    for i in range(4)
                ]

            # Crop the frame using the smoothed coordinates (adjusted to 9:16 aspect ratio)
            cropped_frame = frame[smoothed_crop_coords[1]:smoothed_crop_coords[3], smoothed_crop_coords[0]:smoothed_crop_coords[2]]

            # Resize the cropped frame to the desired 9:16 aspect ratio (output_width x output_height)
            cropped_frame_resized = cv2.resize(cropped_frame, (output_width, output_height))

            # Write the resized cropped frame to the output video
            out.write(cropped_frame_resized)

            # Update the previous crop coordinates
            previous_crop_coords = smoothed_crop_coords

        frame_count += 1

    cap.release()
    out.release()

In [ ]:
def move_to_drive(souce_path, destination_path):
  shutil.move(souce_path, destination_path)

In [ ]:
def add_sound_to_cuted_video(original_video_path, new_video_path, output_path):
    try:
        # Load the original video to extract its audio
        original_video = VideoFileClip(original_video_path)
        if original_video.audio is None:
            raise ValueError("Original video has no audio track")
        original_audio = original_video.audio

        # Load the edited video
        edited_video = VideoFileClip(new_video_path)

        # Get durations
        original_audio_duration = original_audio.duration
        edited_video_duration = edited_video.duration

        # Handle cases based on video lengths
        if original_audio_duration >= edited_video_duration:
            # Trim the original audio to match the edited video
            trimmed_audio = original_audio.subclip(0, edited_video_duration)
            final_video = edited_video.set_audio(trimmed_audio)
        else:
            # Cut the edited video to match the original audio duration
            trimmed_video = edited_video.subclip(0, original_audio_duration)
            final_video = trimmed_video.set_audio(original_audio)

        # Write to a temporary file first
        temp_output_path = output_path + "_temp.mp4"
        final_video.write_videofile(
            temp_output_path,
            codec="libx264",
            audio_codec="aac",
            temp_audiofile="temp-audio.m4a",
            remove_temp=True,
            audio=True,  # Explicitly enable audio
            ffmpeg_params=["-strict", "-2"]  # Add FFmpeg parameters for better compatibility
        )

        # Replace the original file after successful export
        os.replace(temp_output_path, output_path)

        # Clean up by closing the video files
        original_video.close()
        edited_video.close()
        final_video.close()

    except Exception as e:
        print(f"Error processing video: {str(e)}")
        # Clean up if there's an error
        if 'original_video' in locals(): original_video.close()
        if 'edited_video' in locals(): edited_video.close()
        if 'final_video' in locals(): final_video.close()
        if os.path.exists(temp_output_path): os.remove(temp_output_path)
        raise

In [ ]:
def process_in_sequence(input_folder_path, output_folder_path):
  for filename in os.listdir(input_folder_path):
    if filename.endswith(".mp4"):
      print('---------------------------------------')
      print(f"Now working with the {filename}")
      print('---------------------------------------')
      MAIN_INPUT_PATH = os.path.join(input_folder_path, filename)
      MAIN_OUTPUT_PATH = os.path.join(output_folder_path, f"output-{filename}")

      process_video(MAIN_INPUT_PATH)

      detections_list = transform_jsonl(DETECTIONS_FILE_PATH)

      cut_video_with_smoothing(MAIN_INPUT_PATH, detections_list, MAIN_OUTPUT_PATH, output_width=OUTPUT_WIDTH, output_height=OUTPUT_HEIGHT)

      add_sound_to_cuted_video(MAIN_INPUT_PATH, MAIN_OUTPUT_PATH, MAIN_OUTPUT_PATH)

In [ ]:
process_in_sequence(BATCH_INPUT_PATH, BATCH_OUTPUT_PATH)

Streaming output truncated to the last 5000 lines.
32: 640x640 (no detections), 6.3ms
33: 640x640 (no detections), 6.3ms
34: 640x640 (no detections), 6.3ms
35: 640x640 (no detections), 6.3ms
36: 640x640 (no detections), 6.3ms
37: 640x640 (no detections), 6.3ms
38: 640x640 (no detections), 6.3ms
39: 640x640 (no detections), 6.3ms
40: 640x640 (no detections), 6.3ms
41: 640x640 (no detections), 6.3ms
42: 640x640 (no detections), 6.3ms
43: 640x640 (no detections), 6.3ms
44: 640x640 (no detections), 6.3ms
45: 640x640 (no detections), 6.3ms
46: 640x640 (no detections), 6.3ms
47: 640x640 (no detections), 6.3ms
48: 640x640 1 skateboard, 6.3ms
49: 640x640 1 skateboard, 6.3ms
50: 640x640 (no detections), 6.3ms
51: 640x640 (no detections), 6.3ms
52: 640x640 (no detections), 6.3ms
53: 640x640 (no detections), 6.3ms
54: 640x640 (no detections), 6.3ms
55: 640x640 (no detections), 6.3ms
56: 640x640 (no detections), 6.3ms
57: 640x640 (no detections), 6.3ms
58: 640x640 (no detections), 6.3ms
59: 640x64

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-luH-rOYixY8.mp4_temp.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-luH-rOYixY8.mp4_temp.mp4
---------------------------------------
Now working with the W23wqmMco6E.mp4
---------------------------------------
Processed frame 10
Processed frame 20
Processed frame 30
Processed frame 40
Processed frame 50
Processed frame 60
Processed frame 70
Processed frame 80
Processed frame 90
Processed frame 100
Processed frame 110
Processed frame 120

0: 640x640 (no detections), 6.6ms
1: 640x640 (no detections), 6.6ms
2: 640x640 (no detections), 6.6ms
3: 640x640 (no detections), 6.6ms
4: 640x640 1 person, 1 dog, 6.6ms
5: 640x640 1 dog, 6.6ms
6: 640x640 1 person, 1 dog, 1 chair, 6.6ms
7: 640x640 1 cat, 2 dogs, 1 chair, 1 book, 6.6ms
8: 640x640 2 dogs, 1 chair, 1 book, 6.6ms
9: 640x640 2 dogs, 1 chair, 1 book, 6.6ms
10: 640x640 1 person, 1 dog, 1 chair, 1 book, 6.6ms
11: 640x640 1 person, 1 dog, 1 chair, 1 book, 6.6ms
12: 640x640 1 person, 1 dog, 1 chair, 1

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-W23wqmMco6E.mp4_temp.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-W23wqmMco6E.mp4_temp.mp4
---------------------------------------
Now working with the S3r4CFtv9uU.mp4
---------------------------------------
Processed frame 10
Processed frame 20
Processed frame 30
Processed frame 40
Processed frame 50
Processed frame 60
Processed frame 70
Processed frame 80
Processed frame 90
Processed frame 100
Processed frame 110
Processed frame 120

0: 640x640 1 truck, 6.8ms
1: 640x640 1 truck, 6.8ms
2: 640x640 1 truck, 6.8ms
3: 640x640 1 truck, 6.8ms
4: 640x640 1 truck, 6.8ms
5: 640x640 1 truck, 6.8ms
6: 640x640 1 truck, 6.8ms
7: 640x640 1 truck, 6.8ms
8: 640x640 1 truck, 6.8ms
9: 640x640 1 truck, 6.8ms
10: 640x640 1 truck, 6.8ms
11: 640x640 1 truck, 6.8ms
12: 640x640 1 truck, 6.8ms
13: 640x640 1 truck, 6.8ms
14: 640x640 1 truck, 6.8ms
15: 640x640 1 truck, 6.8ms
16: 640x640 1 truck, 6.8ms
17: 640x640 1 truck, 6.8ms
18: 640x640 1 truck, 6.8ms
19: 640x64

MoviePy - Done.
Moviepy - Writing video /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-S3r4CFtv9uU.mp4_temp.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Genuin Assignment/Inference/double_class_ip/output-S3r4CFtv9uU.mp4_temp.mp4
---------------------------------------
Now working with the Kfj1SJBaeuQ.mp4
---------------------------------------
Processed frame 10
Processed frame 20
Processed frame 30
Processed frame 40
Processed frame 50
Processed frame 60
Processed frame 70


KeyboardInterrupt: 